## Personal Project for Section 1

#### Checklist

1) [X] Move inference to Bedrock API & Claude/Nova.
2) [X] Test Multilingual responses (Greek/English) - looks like Nova Lite understands Greek but responds in English.
3) [ ] Add Google Calendar booking tool
4) [X] Implement Bedrock guardrails (instead of evaluation) (needs testing)
5) [ ] Create a better chat interface
6) [X] Improve System Prompt
7) [X] Improve tools - record contact details and questions in DynamoDB (needs testing)
8) [X] Deploy as app (ECS)


In [ ]:
# imports

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr
import boto3
from datetime import datetime
import uuid


In [2]:
# The usual start
os.environ['AWS_BEARER_TOKEN_BEDROCK'] = os.getenv('AWS_BEARER_TOKEN_BEDROCK', 'your-key-if-not-using-env')

region = 'us-east-1' # change to your preferred region - be aware that not all regions have access to all models. If in doubt, use us-east-1.

bedrock = boto3.client(service_name="bedrock", region_name=region) # use this for information and management calls (such as model listings)
bedrock_runtime = boto3.client(service_name="bedrock-runtime", region_name=region) # this is for inference.

# Let's do a quick test to see if works.
# We will list the available models.

response = bedrock.list_foundation_models()
models = response['modelSummaries']
print(f'AWS Region: {region} - Models:')
for model in models:
    print(f"Model ID: {model['modelId']}, Name: {model['modelName']}")

AWS Region: us-east-1 - Models:
Model ID: stability.stable-image-remove-background-v1:0, Name: Stable Image Remove Background
Model ID: stability.stable-image-style-guide-v1:0, Name: Stable Image Style Guide
Model ID: stability.stable-image-control-sketch-v1:0, Name: Stable Image Control Sketch
Model ID: anthropic.claude-sonnet-4-20250514-v1:0, Name: Claude Sonnet 4
Model ID: stability.stable-image-erase-object-v1:0, Name: Stable Image Erase Object
Model ID: stability.stable-image-control-structure-v1:0, Name: Stable Image Control Structure
Model ID: stability.stable-image-search-recolor-v1:0, Name: Stable Image Search and Recolor
Model ID: openai.gpt-oss-120b-1:0, Name: gpt-oss-120b
Model ID: twelvelabs.pegasus-1-2-v1:0, Name: Pegasus v1.2
Model ID: stability.stable-style-transfer-v1:0, Name: Stable Image Style Transfer
Model ID: twelvelabs.marengo-embed-2-7-v1:0, Name: Marengo Embed v2.7
Model ID: stability.stable-image-search-replace-v1:0, Name: Stable Image Search and Replace
Model

In [3]:
# For pushover

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

Pushover user found and starts with u
Pushover token found and starts with a


In [ ]:
# Initialize DynamoDB client
dynamodb = boto3.resource('dynamodb', 
                          region_name='us-east-1', 
                          aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
                          aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY')
)
table = dynamodb.Table('user-contacts')  # Replace with your table name

In [4]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [5]:
push("HEY!!")

Push: HEY!!


In [ ]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    try:
        # Record in DynamoDB
        item = {
            'id': str(uuid.uuid4()),
            'email': email,
            'name': name,
            'notes': notes,
            'timestamp': datetime.utcnow().isoformat(),
            'source': 'website-chat'
        }
        
        table.put_item(Item=item)
        
        # Keep the push notification
        push(f"Recording interest from {name} with email {email} and notes {notes}")
        
        return {"recorded": "ok", "id": item['id']}
        
    except Exception as e:
        # Fallback to just notification if DynamoDB fails
        push(f"Recording interest from {name} with email {email} and notes {notes}")
        return {"recorded": "notification_only", "error": str(e)}

In [7]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

In [8]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "inputSchema": {
        "json": {
            "type": "object",
            "properties": {
                "email": {
                    "type": "string",
                    "description": "The email address of this user"
                },
                "name": {
                    "type": "string",
                    "description": "The user's name, if they provided it"
                }
                ,
                "notes": {
                    "type": "string",
                    "description": "Any additional information about the conversation that's worth recording to give context"
                }
            },
            "required": ["email"],
            "additionalProperties": False
        }
    }
}

In [9]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "inputSchema": { 
        "json": {
            "type": "object",
            "properties": {
                "question": {
                    "type": "string",
                    "description": "The question that couldn't be answered"
                },
            },
            "required": ["question"],
            "additionalProperties": False
        }
    }
}

In [10]:
tools = {
        "tools": 
            [{"toolSpec": record_user_details_json},
            {"toolSpec": record_unknown_question_json}
            ]
}

In [11]:
tools

{'tools': [{'toolSpec': {'name': 'record_user_details',
    'description': 'Use this tool to record that a user is interested in being in touch and provided an email address',
    'inputSchema': {'json': {'type': 'object',
      'properties': {'email': {'type': 'string',
        'description': 'The email address of this user'},
       'name': {'type': 'string',
        'description': "The user's name, if they provided it"},
       'notes': {'type': 'string',
        'description': "Any additional information about the conversation that's worth recording to give context"}},
      'required': ['email'],
      'additionalProperties': False}}}},
  {'toolSpec': {'name': 'record_unknown_question',
    'description': "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    'inputSchema': {'json': {'type': 'object',
      'properties': {'question': {'type': 'string',
        'description': "The question that couldn't be answered"}},
      'requ

In [ ]:
# This function can take a list of tool calls, and run them. This is the IF statement!!

# def handle_tool_calls(tool_calls):
#    results = []
#    for tool_call in tool_calls:
#        tool_name = tool_call.function.name
#        arguments = json.loads(tool_call.function.arguments)
#        print(f"Tool called: {tool_name}", flush=True)
#
#       # THE BIG IF STATEMENT!!!
#
#        if tool_name == "record_user_details":
#            result = record_user_details(**arguments)
#        elif tool_name == "record_unknown_question":
#            result = record_unknown_question(**arguments)
#
#        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
#   return results

In [17]:
globals()["record_unknown_question"]("this is a really hard question")

Push: Recording this is a really hard question asked that I couldn't answer


{'recorded': 'ok'}

In [19]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Eleftherios Chaniotakis"

In [28]:
# Add today's date to the system prompt (reduces chances of LLM responding that we're in the future)

today = datetime.today().strftime('%Y-%m-%d')

system_prompt = f"""Today's date is {today}.

You are {name}'s professional AI representative on their website.

## Your Role:
- Answer questions about {name}'s career, skills, experience, and professional background
- Maintain a professional, engaging tone suitable for potential clients and employers
- Stay strictly within professional topics - redirect personal or off-topic questions

## Key Behaviors:
1. **Professional Focus**: Only discuss career-related topics, technical skills, work experience, and professional achievements
2. **Lead Generation**: When users show interest, guide them toward providing contact information
3. **Knowledge Gaps**: Use the record_unknown_question tool for ANY question you cannot answer
4. **Contact Collection**: Use record_user_details tool when users provide email addresses

## Response Guidelines:
- Keep responses concise and relevant
- Highlight {name}'s key strengths and expertise
- Ask follow-up questions to understand visitor needs
- Suggest next steps (portfolio review, consultation, etc.)

## Available Information:
### Summary:
{summary}

### LinkedIn Profile:
{linkedin}

Represent {name} authentically and professionally. Focus on their value proposition and encourage meaningful professional connections."""

system_prompt = [{"text": system_prompt}]

BEDROCK_MODEL_ID = 'us.amazon.nova-pro-v1:0'  # try "us.amazon.nova-lite-v1:0" for faster responses.
messages=[]

# Amazon Bedrock Guardrails configuration
# First create a guardrail in AWS Console
# Then configure a policy. Choose:
# - Hate & Toxicity: Enable with HIGH blocking strength
# - Denied topics: Add "Politics", "Personal opinions", "Controversial topics"
# then replace the guardrail ID with the actual guardrail identifier.

guardrails_config = {
    "guardrailIdentifier": "your-guardrail-id",  # Replace with your guardrail ID
    "guardrailVersion": "DRAFT"
}

In [ ]:
def convert_openai_to_bedrock(history):
    """Convert OpenAI chat format to Bedrock Converse API format"""
    bedrock_messages = []
    
    for message in history:
        if message["role"] == "system":
            continue  # Handle system messages separately
        elif message["role"] in ["user", "assistant"]:
            bedrock_messages.append({
                "role": message["role"],
                "content": [{"text": message["content"]}]
            })
    
    return bedrock_messages

def handle_tool_calls(tool_call):
    """Handle single Bedrock tool call"""
    tool_name = tool_call['name']
    arguments = tool_call['input']  # Already a dict in Bedrock
    print(f"Tool called: {tool_name} with arguments {arguments}", flush=True)
    
    tool = globals().get(tool_name)
    result = tool(**arguments) if tool else {}
    
    return {
        "role": "user",
        "content": [{
            "toolResult": {
                "toolUseId": tool_call['toolUseId'],
                "content": [{"json": result}]
            }
        }]
    }

def chat(message, history):
    # Convert history to Bedrock format
    messages = convert_openai_to_bedrock(history)
    
    # Add current message
    messages.append({
        "role": "user", 
        "content": [{"text": message}]
    })
    
    done = False
    while not done:
        response = bedrock_runtime.converse(
            modelId=BEDROCK_MODEL_ID, 
            messages=messages, 
            system=system_prompt,
            toolConfig=tools,
            guardrailConfig=guardrails_config  # Add guardrails
        )

        # Check for guardrail intervention
        if 'trace' in response and 'guardrail' in response['trace']:
            guardrail_trace = response['trace']['guardrail']
            if guardrail_trace.get('action') == 'GUARDRAIL_INTERVENED':
                return "I can only discuss professional topics related to my career and experience. Please ask about my work, skills, or professional background."

        output_message = response['output']['message']
        messages.append(output_message)
        finish_reason = response['stopReason']
        
        if finish_reason == "tool_use":
            tool_requests = output_message['content']
            for tool_request in tool_requests:
                if 'toolUse' in tool_request:
                    tool_result = handle_tool_calls(tool_request['toolUse'])
                    messages.append(tool_result)
        else:
            done = True
    
    return output_message['content'][0]['text']

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Tool called: record_user_details with arguments {'email': 'user@example.com'}
Push: Recording interest from Name not provided with email user@example.com and notes not provided


## And now for deployment

This code is in `app.py`

We will deploy to HuggingFace Spaces.

Before you start: remember to update the files in the "me" directory - your LinkedIn profile and summary.txt - so that it talks about you! Also change `self.name = "Ed Donner"` in `app.py`..  

Also check that there's no README file within the 1_foundations directory. If there is one, please delete it. The deploy process creates a new README file in this directory for you.

1. Visit https://huggingface.co and set up an account  
2. From the Avatar menu on the top right, choose Access Tokens. Choose "Create New Token". Give it WRITE permissions - it needs to have WRITE permissions! Keep a record of your new key.  
3. In the Terminal, run: `uv tool install 'huggingface_hub[cli]'` to install the HuggingFace tool, then `hf auth login` to login at the command line with your key. Afterwards, run `hf auth whoami` to check you're logged in  
4. Take your new token and add it to your .env file: `HF_TOKEN=hf_xxx` for the future
5. From the 1_foundations folder, enter: `uv run gradio deploy` 
6. Follow its instructions: name it "career_conversation", specify app.py, choose cpu-basic as the hardware, say Yes to needing to supply secrets, provide your openai api key, your pushover user and token, and say "no" to github actions.  

Thank you Robert, James, Martins, Andras and Priya for these tips.  
Please read the next 2 sections - how to change your Secrets, and how to redeploy your Space (you may need to delete the README.md that gets created in this 1_foundations directory).

#### More about these secrets:

If you're confused by what's going on with these secrets: it just wants you to enter the key name and value for each of your secrets -- so you would enter:  
`OPENAI_API_KEY`  
Followed by:  
`sk-proj-...`  

And if you don't want to set secrets this way, or something goes wrong with it, it's no problem - you can change your secrets later:  
1. Log in to HuggingFace website  
2. Go to your profile screen via the Avatar menu on the top right  
3. Select the Space you deployed  
4. Click on the Settings wheel on the top right  
5. You can scroll down to change your secrets (Variables and Secrets section), delete the space, etc.

#### And now you should be deployed!

If you want to completely replace everything and start again with your keys, you may need to delete the README.md that got created in this 1_foundations folder.

Here is mine: https://huggingface.co/spaces/ed-donner/Career_Conversation

I just got a push notification that a student asked me how they can become President of their country 😂😂

For more information on deployment:

https://www.gradio.app/guides/sharing-your-app#hosting-on-hf-spaces

To delete your Space in the future:  
1. Log in to HuggingFace
2. From the Avatar menu, select your profile
3. Click on the Space itself and select the settings wheel on the top right
4. Scroll to the Delete section at the bottom
5. ALSO: delete the README file that Gradio may have created inside this 1_foundations folder (otherwise it won't ask you the questions the next time you do a gradio deploy)


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">• First and foremost, deploy this for yourself! It's a real, valuable tool - the future resume..<br/>
            • Next, improve the resources - add better context about yourself. If you know RAG, then add a knowledge base about you.<br/>
            • Add in more tools! You could have a SQL database with common Q&A that the LLM could read and write from?<br/>
            • Bring in the Evaluator from the last lab, and add other Agentic patterns.
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">Aside from the obvious (your career alter-ego) this has business applications in any situation where you need an AI assistant with domain expertise and an ability to interact with the real world.
            </span>
        </td>
    </tr>
</table>